In [ ]:
import random
import numpy as np
from scipy.io.wavfile import write
import IPython.display as ipd
import math

In [ ]:
def melody_generation(tone = 440, key = 'minor', duration = 60, bpm = 120, wave_type = 'sine', sample_rate = 44100):

    
    ## Создание цепочки аккордов для минорной тональности (по тонам):
    if key == 'minor':
        second, third = random.sample([3, 4, 6, 7], 2)
        chords_chain = [1, second, third, 5] 
    
                                                ## Пример: chords_chain = [1, 3, 7, 2]

    ## Создание цепочки аккордов для мажорной тональности (по тонам): (???????????????????????????????????????????????????????)
    if key == 'major':
        second, third = random.sample([3, 4, 6, 7], 2)
        chords_chain = [1, second, third, 5] 
    
                                                ## Пример: chords_chain = [1, 3, 7, 2]
    
    ## Преобразование ступеней цепочки в частоты:
    
    major_intervals = [0, 2, 4, 5, 7, 9, 11]
    minor_intervals = [0, 2, 3, 5, 7, 8, 10]

        ## Функция для поиска частоты:
    def note_frequency(tone, step):
        a = 2 ** (1 / 12)
        return tone * (a ** step)

    second = chords_chain[1] - 1
    third = chords_chain[2] - 1
    
    dominant = note_frequency(tone, 7)
    
    if key == 'major':
        second = major_intervals[second]
        third =  major_intervals[third]
        
    elif key == 'minor':
        second = minor_intervals[second]
        third =  minor_intervals[third]      

    second = note_frequency(tone, second)
    third = note_frequency(tone, second)
    
    frequencies = [tone, second, third, dominant]
    
                                                ## Пример: frequencies = [164.81, 293.6580354734188, 219.99495636576341, 246.93598934004595]
    

    
    ## Сравнение частот цепочки и транспонирование при большом расстоянии
    
    def adjust_to_tone(value, tone):
        best_value = value
        smallest_distance = abs(value - tone)
        divided_value = value
        multiplied_value = value
        
        while divided_value >= tone or multiplied_value <= tone * 2:
            divided_value /= 2
            multiplied_value *= 2
            
            for val in [divided_value, multiplied_value]:
                distance = abs(val - tone)
                if distance < smallest_distance:
                    best_value = val
                    smallest_distance = distance
                    
        return best_value
    
    for i in range(1, len(frequencies)):
        frequencies[i] = adjust_to_tone(frequencies[i], tone)
        
                                                ## Пример до: frequencies = [164.81, 293.6580354734188, 219.99495636576341, 246.93598934004595]
                                                ##     после: frequencies = [164.81, 146.8290177367094, 109.99747818288171, 123.46799467002297]


    
    ## Создание звуковой волны для цепочки аккордов:

    chain_duration = 4 / (bpm / duration)

    time = np.linspace(0, chain_duration, int(44100 * chain_duration), endpoint=False)

        ## Функции, генерирующие разные волны:
    def sine(frequency, amplitude=0.5):
        wave = amplitude * np.sin(2 * np.pi * frequency * time)
        return wave
    def saw(frequency, amplitude=0.5):
        period = 1 / frequency
        phase = (time / period) % 1
        wave = (amplitude * 2 * phase) - amplitude
        return wave 
    def square(frequency, amplitude=0.5):
        wave = amplitude * np.sign(np.sin(2 * np.pi * frequency * time))
        return wave        
    def triangle(frequency, amplitude=0.5):
        wave = amplitude * (2 * np.abs(2 * (time * frequency - np.floor(time * frequency + 0.5))) - 1)
        return wave

        ## Генерация мажорного и минорного трезвучий:
    chord_duration = chain_duration/4
    wave_functions = {
        'sine': sine,
        'saw': saw,
        'square': square,
        'triangle': triangle
    }
    def major_triad(tone, chord_duration):
        
        mediant = note_frequency(tone, 4)
        dominant = note_frequency(tone, 7)
        frequencies = [tone, mediant, dominant]
        
        waves = [wave_functions[wave_type](freq) for freq in frequencies]
        
        wave = np.sum(wave, axis=0)
        wave /= np.max(np.abs(wave))
        wave = np.int16(wave * 32767)
        return wave

    def minor_triad(tone, chord_duration):
        
        mediant = note_frequency(tone, 4)
        dominant = note_frequency(tone, 7)
        frequencies = [tone, mediant, dominant]
        
        wave = [wave_functions[wave_type](freq) for freq in frequencies]
        
        wave = np.sum(wave, axis=0)
        wave /= np.max(np.abs(wave))
        wave = np.int16(wave * 32767)
        return wave

        ## Функция для создания звуковой волны аккомпанемента 
    def create_chord_wave(tone, duration, key, chords_chain):
        i = len(chain)
        chords = chords_chain

        if key == 'major':
            chord_1 = mor_triad(tone, duration/i)
            chord_2 = mor_triad(chords[1], chain_duration/i)
            chord_3 = mor_triad(chords[2], chain_duration/i)
            chord_4 = mor_triad(chords[3], chain_duration/i)
            
        else:
            chord_1 = minor_triad(tone, duration/i)
            if chain[1] == 4:
                chord_2 = minor_triad(chords[1], chain_duration/i)
            else:
                chord_2 = major_triad(chords[1], chain_duration/i)
            if chain[2] == 4:
                chord_3 = minor_triad(chords[2], chain_duration/i)
            else:
                chord_3 = major_triad(chords[2], chain_duration/i)
            chord_4 = major_triad(chords[3], chain_duration/i)
            
        chain_wave = np.concatenate((chord_1, chord_2, chord_3, chord_4))

        accompaniment = np.tile(chain_wave, duration // chain_duration)
        
        if duration % chain_duration > 0:
            accompaniment = np.concatenate((accompaniment, chain_wave[:remaining_samples]))
        
        write('accompaniment.wav', sample_rate, accompaniment)
        return (chain_wave, accompaniment)
                                                ## Результат в конце create_chord_wave это сохранение файла с аккомпанементом,
                                                ## сохранение волны всего аккомпанемента и волны цепочки аккордов
                                                     
    ## Функция для создания ритмического рисунка для мелодии (0.125-1 это длительность ноты):
    def rhythmic_pattern():
        values = [0.125, 0.25, 0.5, 0.75, 1]
                                                                    ##[0.125, 0.25, 0.375, 0.5, 0.625, 0.75, 0.875, 1]
        weights = [5, 4, 3, 2, 1]  
        pattern = []
        current_sum = 0
        
        while current_sum < 1:
            value = random.choices(values, weights)[0]
            if current_sum + value <= 1:
                pattern.append(value)
                current_sum += value
        random.shuffle(pattern)
        return pattern
                                                ## Пример: pattern = [0.125, 0.375, 0.375, 0.125]


    ## Функция для создания массива с ритмическим рисунком и соответственные частоты нот

    def chord_step(start):
        values = []
        current_value = start
        for _ in range(3):
            values.append(current_value)
            current_value += 2
            if current_value > 7:
                current_value = current_value % 7  
                if current_value == 0:
                    current_value = 7
        return values
    
    def chord_st_chain(input_list):
        results = {}
        for i, value in enumerate(input_list):
            results[f"ch_{i+1}"] = chord_step(value)
        return results
        
    result = chord_st_chain(chords_chain)
 
    ## Функция для генерации мелодии:
    def generate_melody(chord_st_chain):
        all_steps = []
        all_durations = []
    
        for notes in chord_st_chain.values():
            rythm = rhythmic_pattern()
            melody_notes = [random.choice(notes) for _ in rythm]
            all_steps.extend(melody_notes)
            all_durations.extend(rythm)
        
        return [all_steps, all_durations]

    melody = generate_melody(result)



    ## Функция для преобразования ступеней в частоты 
    def melody_freq(melody, tone):
        
        if key == 'major':
            intervals = [0, 2, 3, 5, 7, 8, 10]
        else:
            intervals = [0, 2, 3, 5, 7, 8, 10]
            
        steps = melody[0]
        durations = melody[1]
    
        frequencies = [note_frequency(tone, intervals[step - 1]) for step in steps]

         
        return [frequencies, durations]
    
    melody_freq = melody_freq(melody, tone)

    ## Функция для создания звуковой волны мелодии
    def create_melody_wave():
        num_repeats = duration // chain_duration
        remaining_time = duration % base_duration
        remaining_samples = int(remaining_time * semple_rate)
        
        base_melody = generate_melody(result=None)
        melody = np.tile(base_melody, int(num_repeats))
        
        if remaining_samples > 0:
            melody = np.concatenate((melody, base_melody[:remaining_samples]))
        write('melody.wav', sample_rate, melody)
        return melody
                                                ## Результат в конце create_melody_wave это сохранение файла с мелодией
                                                ## и сохранение волны мелодии 
    ## Объединение мелодии и аккомпанемента:
    final_wave = 'melody.wav' + 'accompaniment.wav'
    final_wave = np.int16(final_wave * 32767)
    write('music.wav', sample_rate, final_wave)
    return (ipd.Audio('music.wav'))



melody_generation()